In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

# Загрузка изображения
img_color = cv2.imread('zenitsu.jpg', cv2.IMREAD_COLOR)
img_gray = cv2.imread('zenitsu.jpg', cv2.IMREAD_GRAYSCALE)

# Отображение изображений
print("Цветное изображение:")
cv2_imshow(img_color)

print("Черно-белое изображение:")
cv2_imshow(img_gray)

# Сохранение черно-белого изображения
cv2.imwrite('gray_zenitsu.jpg', img_gray)

# Размеры изображения
print(f"Высота: {img_color.shape[0]}")
print(f"Ширина: {img_color.shape[1]}")
print(f"Количество каналов: {img_color.shape[2]}")

# Получаем значение пикселя (0,0)
(b, g, r) = img_color[0, 0]
print(f"Пиксель (0,0): Красный={r}, Зелёный={g}, Синий={b}")

# Изменяем пиксель (0,0)
img_color[0, 0] = (255, 0, 0)  # BGR — синий
(b, g, r) = img_color[0, 0]
print(f"Новое значение пикселя (0,0): Красный={r}, Зелёный={g}, Синий={b}")

# Изменение размера
resized_img = cv2.resize(img_color, (200, 200), interpolation=cv2.INTER_LINEAR)

# Отображение
print("Изменённое изображение:")
cv2_imshow(resized_img)

# Сохранение
cv2.imwrite('resized_zenitsu.jpg', resized_img)

# Создание матрицы сдвига
tx, ty = 50, 100
M = np.float32([[1, 0, tx], [0, 1, ty]])

# Применение сдвига
translated_img = cv2.warpAffine(img_color, M, (img_color.shape[1], img_color.shape[0]))

# Отображение
print("Сдвинутое изображение:")
cv2_imshow(translated_img)

# Обрезаем область (x1=300, y1=10, x2=750, y2=450)
cropped_img = img_color[10:450, 300:750]

# Отображение и сохранение
print("Обрезанное лицо:")
cv2_imshow(cropped_img)
cv2.imwrite('cropped_zenitsu.jpg', cropped_img)

# Параметры поворота
center = (img_color.shape[1] // 2, img_color.shape[0] // 2)
angle = 45
scale = 0.6

# Матрица поворота
M = cv2.getRotationMatrix2D(center, angle, scale)

# Применение поворота
rotated_img = cv2.warpAffine(img_color, M, (img_color.shape[1], img_color.shape[0]))

# Отображение
print("Повёрнутое изображение:")
cv2_imshow(rotated_img)

# Матрица для изменения яркости
M = np.ones(img_color.shape, dtype="uint8") * 50

# Сложение и вычитание
brightened = cv2.add(img_color, M)
darkened = cv2.subtract(img_color, M)

# Отображение
print("Увеличенная яркость:")
cv2_imshow(brightened)

print("Уменьшенная яркость:")
cv2_imshow(darkened)

# Размытие: усреднение
blurred_avg = cv2.blur(img_color, (5, 5))

# Гауссовское размытие
blurred_gauss = cv2.GaussianBlur(img_color, (5, 5), 0)

# Медианное размытие
blurred_median = cv2.medianBlur(img_color, 5)

# Отображение
print("Усреднённое размытие:")
cv2_imshow(blurred_avg)

print("Гауссовское размытие:")
cv2_imshow(blurred_gauss)

print("Медианное размытие:")
cv2_imshow(blurred_median)

# Загрузка основного изображения
cards_img = cv2.imread('cards.png')

# Преобразование в оттенки серого и размытие
gray_cards = cv2.cvtColor(cards_img, cv2.COLOR_BGR2GRAY)
blurred_cards = cv2.GaussianBlur(gray_cards, (3, 3), 0)

# Бинаризация изображения
_, thresh = cv2.threshold(blurred_cards, 215, 255, cv2.THRESH_BINARY)

# Поиск контуров
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Фильтрация и обрезка карт
card_images = []
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    if w > 40 and h > 50:  # фильтр по размеру
        card = cards_img[y:y+h, x:x+w]
        card_images.append(card)
        cv2.rectangle(cards_img, (x, y), (x + w, y + h), (0, 0, 255), 2)

print("Распознанные карты:")
cv2_imshow(cards_img)

# Сохраняем обрезанные карты (опционально)
for i, card in enumerate(card_images):
    cv2.imwrite(f'./cards/card_{i + 1}.png', card)

# Функция для сравнения двух изображений
def match_card(template, image):
    # Инициализация ORB детектора
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(template, None)
    kp2, des2 = orb.detectAndCompute(image, None)

    if des1 is None or des2 is None:
        return 0

    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
    matches = bf.knnMatch(des1, des2, k=2)

    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    return len(good_matches)

# Загрузка эталонного изображения
template_king = cv2.imread('king_card.png', 0)

# Распознавание карт
print("\nСравнение с шаблоном 'Король':")
for idx, card in enumerate(card_images):
    gray_card = cv2.cvtColor(card, cv2.COLOR_BGR2GRAY)
    matches = match_card(template_king, gray_card)
    print(f"Карта {idx+1}: найдено совпадений с королём — {matches}")